# Investigating the Best Strategy for Portfolio Optimisation as a Zambia Citizen 


This project investigates optimal investment strategies for Zambian citizens facing unique economic challenges including high inflation and currency volatility. Using Monte Carlo simulation and modern portfolio theory, it analyzes four distinct asset classes to determine the best risk-adjusted portfolio allocation for Zambian investors.

Portfolio management, to me, is about turning uncertainty into disciplined decisions: diversify across global assets, respect tail risk, and measure costs honestly. 

## Project Overview

This project seeks to answer: What is the optimal global asset allocation strategy for Zambian investors to maximize risk-adjusted returns while mitigating local economic risks?



## Methodology

Data Collection: Collected 10 years of historical data (2015-2025) using Yahoo Finance API

Four asset classes:

- ZMW/USD exchange rate (ZMW=X)
- Copper futures (HG=F) - relevant to Zambia's copper-dependent economy
- S&P 500 index (^GSPC) - global equity exposure
- Gold futures (GC=F) - safe haven asset


I have choosen 4 distinct categories - ones that relate to zambia Inflation and commodity cycles and two that are independent of Zambia's Inflation and commodity cycles. To test whether it is better to invest in assets outside my home country.



In [ ]:
# Import Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import yfinance as yf
from scipy.optimize import minimize

In [ ]:
tickers = [
    "ZMW=X",
    "HG=F", # Copper futures (reseach what futures are)
    "^GSPC", # S&P 500
    "GC=F"   # Gold futures 
]




In [ ]:
#  Download 10 years of daily data till date

data = yf.download(tickers, start="2015-01-01", end="2025-08-01")["Close"]

# Clean the data , remove null and missing data

data = data.dropna()
print(data.head())


## How do we quantify a good investment?  

In finance there are several ways to go about `measuring risk`. In this project, I will use  the `Sharpe Ratio`. 

**Sharpe Ratio**:  is defined as the measure of risk-adjusted returns 
Risk-Adjusted Performance Metric
The primary measure for evaluating investment performance in this analysis is the risk-adjusted return metric, calculated as follows:

`Performance Measure = E[P − B] / D`

**In this formulation**:

- P represents the total return of the investment portfolio
- B denotes the baseline return of a risk-free security
- D signifies the variability of the portfolio's returns above the baseline

**Portfolio Return Calculation**
The overall portfolio return is determined by combining individual asset returns using this approach:

`Total Return = (s₁ × a₁) + (s₂ × a₂) + ... + (sₙ × aₙ)`

Where for each investment component:

- a indicates the periodic return of each asset
- s represents the proportional allocation to each asset

**Portfolio Volatility Measurement**

The overall portfolio variability is computed considering individual asset risks and their interrelationships:

`Overall Variability = √[ (s₁²v₁²) + (s₂²v₂²) + (2s₁s₂c₁₂v₁v₂) + ... ]`

Where:

v represents the individual volatility of each asset's returns
s indicates the allocation percentage for each asset
c measures the co-movement between different asset pairs


Based on the foundational work of William Sharpe regarding risk-adjusted performance measurement

Reference: https://web.stanford.edu/~wfsharpe/art/sr/sr.htm


In [ ]:
# ----------------------
# PROCESSING THE DATA 
# ----------------------

returns = data.pct_change().dropna() # daily percentage returns 
mean_returns = returns.mean() * 252 # annulised average returns 
covariance_matrix = returns.cov() * 252 # annulised covariance between tickers 

print(f"Mean Returns: {mean_returns}")
print(f"Covariance Matrix: {covariance_matrix}")

## WHY USE MONTE CARLO FOR PORTFOLIO OPTIMISATION

Monte carlo is a numerical method used to appoximate solution through random modeling. 

 Monte carlo is an appropriate tool to use for portfolio optimisation as it is able to capture `uncertainity and variablity` that are inherent in financial markets. 

In Finance, Monte carlo generates thousands of possible future events by randomly sampling from the portfolio return distribution which allows investors to study the distribution outcome rather single-point estimation



In [ ]:
# --------------------
# Monte Carlo Simulation
# --------------------

number_of_portfolios = 10000
results_matrix = np.zeros((3, number_of_portfolios)) # return, volatility, Sharpe ratio 
weights_array = []
risk_free_rate = .1952

for i in range(number_of_portfolios): 
    weights = np.random.random(len(tickers))
    weights /= sum(weights)
    weights_array.append(weights)

    portfolio_returns = np.dot(weights,mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

    #The yield on Zambia Government Bond 10y held steady at 19.52% on August 22, 2025.
    sharpe_ratio = (portfolio_returns - risk_free_rate)/portfolio_volatility

    results_matrix[0,i] = portfolio_returns
    results_matrix[1,i] = portfolio_volatility
    results_matrix[2, i] = sharpe_ratio



## GRAPHICAL REPRESENTATION



In [ ]:

plt.figure(figsize=(10,6))
plt.scatter(x=results_matrix[1,:],y=results_matrix[0,:], c=results_matrix[2,:], cmap='viridis', s=10)
plt.colorbar(label="Sharpe Ratio")
plt.xlabel("Volatility")
plt.ylabel("Expected Return")
plt.title("Monte Carlo Porfolio Optimisation - Focused On Zambia")
max_sharpe_index = np.argmax(results_matrix[2])
plt.scatter(results_matrix[1,max_sharpe_index], results_matrix[0,max_sharpe_index], c="red", s=50, marker='*', label= "Max Sharpe")
plt.legend()
plt.show()

## MOST FAVORABLE WEIGHTS

In [ ]:

weights_df = pd.DataFrame(weights_array, columns=tickers)
best_weights= weights_df.iloc[max_sharpe_index]
print("Best Portfolio Weights:")
print(best_weights)


## Portfolio Optimization with SLSQP

We use constrained numerical optimization (SLSQP algorithm) to precisely identify the maximum Sharpe ratio portfolio. This method is fundamentally different from Monte Carlo simulation:

**Key Advantages**:
- Precision: Finds the exact mathematical optimum rather than approximating through random sampling
- Efficiency: Requires significantly fewer computations than Monte Carlo methods
- Constraint Handling: Systematically incorporates investment constraints (no short selling, full allocation)

**How It Works**:

1. We maximize the Sharpe ratio by minimizing its negative value
2. The algorithm calculates gradients to efficiently navigate the solution space
3. It enforces that all weights sum to 1 and remain between 0-1
4. The process converges to the optimal risk-adjusted portfolio

This approach delivers the exact optimal asset allocation that maximizes risk-adjusted returns given the specified constraints and market assumptions.


In [ ]:
# Functions 

def portfolio_performance(weights, mean_return, covariance_matrix, risk_free_rate): 
    op_portfolio_returns = np.dot(weights, mean_return)
    op_portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    op_sharpe_ratio = (op_portfolio_returns - risk_free_rate) / op_portfolio_volatility

    return op_portfolio_returns, op_portfolio_volatility, op_sharpe_ratio

def negative_sharpe_ratio(weights, mean_return, covariance_matrix, risk_free_rate): 
    return -portfolio_performance(weights, mean_return, covariance_matrix, risk_free_rate)[2]

def portfolio_volatility(weights, mean_return, covariance_matrix, risk_free_rate): 
    return portfolio_performance(weights, mean_return, covariance_matrix, risk_free_rate)[1]

# weights must sum to 1 
constraints = ({'type':'eq', 'fun': lambda x: np.sum(x)-1})

# weights must be between 0 to 1 
bounds = tuple((0,1) for _ in range(len(tickers)))
initial_guess = len(tickers) * [1./len(tickers)]

In [ ]:

# Optimize for Max Sharpe Ratio

max_shape_ratio = minimize(negative_sharpe_ratio, initial_guess, 
                           args=(mean_returns, covariance_matrix, risk_free_rate),
                            method='SLSQP', bounds=bounds, constraints=constraints)

op_sharpe_return, op_sharpe_volalitlity, op_sharpe_ratio = portfolio_performance(max_shape_ratio.x, mean_returns, covariance_matrix, risk_free_rate)

print("Max Sharpe Ratio Portfolio:")
print("Weights:\n", dict(zip(tickers, max_shape_ratio.x)))
print("Return:\n", op_sharpe_return)
print("Volatility:\n", op_sharpe_volalitlity)
print("Sharpe Ratio:\n", op_sharpe_ratio)